In [ ]:
!pip install selenium webdriver-manager beautifulsoup4 pandas

In [ ]:
!pip list

In [16]:
import sys, selenium
print(sys.executable)
print(selenium.__version__)

/home/hyuksu/teamproject2/.venv/bin/python
4.40.0


In [17]:
from selenium import webdriver

opt = webdriver.ChromeOptions()
opt.add_argument("--headless=new")
opt.add_argument("--no-sandbox")
opt.add_argument("--disable-dev-shm-usage")
opt.add_argument("--disable-gpu")
opt.binary_location = "/usr/bin/google-chrome"

driver = webdriver.Chrome(options=opt)  # ✅ Selenium Manager가 드라이버 자동 처리
driver.get("https://example.com")
print("TITLE:", driver.title)
driver.quit()


TITLE: Example Domain


In [18]:
from selenium import webdriver

opt = webdriver.ChromeOptions()
opt.add_argument("--headless=new")
opt.add_argument("--no-sandbox")
opt.add_argument("--disable-dev-shm-usage")
opt.add_argument("--disable-gpu")
opt.binary_location = "/usr/bin/google-chrome"

driver = webdriver.Chrome(options=opt)
driver.get("https://rollcall.com/factbase/trump/search/")
html = driver.page_source

print("TITLE:", driver.title)
print("LEN HTML:", len(html))
print("HAS View Transcript?:", "View Transcript" in html)
print("HAS Sign in?:", ("Sign in" in html) or ("sign in" in html))
print("HAS Access denied/Attention?:", ("Access denied" in html) or ("Attention Required" in html))

with open("debug_rollcall.html", "w", encoding="utf-8") as f:
    f.write(html)

driver.quit()
print("saved debug_rollcall.html")


TITLE: Roll Call Factba.se - Donald J. Trump Transcript Search
LEN HTML: 732977
HAS View Transcript?: True
HAS Sign in?: False
HAS Access denied/Attention?: False
saved debug_rollcall.html


In [20]:
from selenium import webdriver

opt = webdriver.ChromeOptions()
opt.add_argument("--headless=new")
opt.add_argument("--no-sandbox")
opt.add_argument("--disable-dev-shm-usage")
opt.binary_location = "/usr/bin/google-chrome"

d = webdriver.Chrome(options=opt)
d.get("https://example.com")
print("OK-1 TITLE:", d.title)
d.quit()


OK-1 TITLE: Example Domain


In [22]:
!curl -I https://rollcall.com/factbase/trump/search/ | head


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
HTTP/2 200 
date: Thu, 22 Jan 2026 07:12:17 GMT
content-type: text/html; charset=UTF-8
server: nginx/1.18.0 (Ubuntu)
vary: Accept-Encoding
link: <https://rollcall.com/wp-json/>; rel="https://api.w.org/"
link: <https://rollcall.com/wp-json/wp/v2/pages/749104>; rel="alternate"; title="JSON"; type="application/json"
link: <https://rollcall.com/?p=749104>; rel=shortlink
x-frame-options: SAMEORIGIN
x-xss-protection: 1; mode=block


In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_all_links():
    url = "https://rollcall.com/factbase/trump/search/"

    # --- [1. 최적화 옵션 설정] ---
    opt = webdriver.ChromeOptions()
    opt.add_argument("--headless=new") 
    opt.add_argument("--no-sandbox")
    opt.add_argument("--disable-dev-shm-usage")
    opt.add_argument("--disable-gpu")
    opt.add_argument("--window-size=1920,1080")
    
    # 이미지 로딩 차단 (속도 UP, 메모리 절약)
    opt.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
    opt.add_argument("--disable-extensions")
    
    # 봇 탐지 회피용 헤더
    opt.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36")
    
    # 페이지 로딩 전략: eager (뼈대만 로딩되면 진행)
    opt.page_load_strategy = 'eager'

    print("🚀 [전체 수집 모드] 브라우저 실행 중...")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opt)

    all_links = []

    try:
        # 접속
        print(f"🌐 사이트 접속: {url}")
        driver.get(url)
        time.sleep(5) # 초기 로딩 대기

        # --- [2. 무한 스크롤 로직] ---
        last_height = driver.execute_script("return document.body.scrollHeight")
        scroll_count = 0
        
        # 안전장치: 최대 스크롤 횟수 (필요하면 1000으로 늘리세요)
        # 보통 1번 스크롤에 10~20개씩 로딩됩니다. 300번이면 약 3000~6000개.
        max_scrolls = 300 
        
        print("⬇️ 전체 데이터를 위해 스크롤을 내립니다...")

        while scroll_count < max_scrolls:
            # 바닥까지 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            
            # 로딩 대기 (인터넷 속도에 따라 조절, 2~3초 권장)
            time.sleep(2.5)
            
            # 높이 변화 체크
            new_height = driver.execute_script("return document.body.scrollHeight")
            
            if scroll_count % 10 == 0:
                print(f"   ... 스크롤 {scroll_count}회 완료 (현재 높이: {new_height})")

            if new_height == last_height:
                # 높이가 안 변하면 데이터가 끝났다는 뜻
                # 혹시 모르니 3초 더 기다리고 한 번 더 체크 (더블 체크)
                time.sleep(3)
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    print("🎉 더 이상 로딩될 데이터가 없습니다! (바닥 도달)")
                    break
            
            last_height = new_height
            scroll_count += 1

        # --- [3. 전체 데이터 추출 (BeautifulSoup)] ---
        print("\n🔍 전체 HTML 분석 및 링크 추출 중...")
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        buttons = soup.select('a[title="View Transcript"]')
        
        for btn in buttons:
            href = btn.get('href')
            if href:
                if "rollcall.com" not in href:
                    href = "https://rollcall.com" + href
                
                # 중복 없이 저장
                if href not in all_links:
                    all_links.append(href)

        print(f"✅ 최종 수집 완료: 총 {len(all_links)}개")

    except Exception as e:
        print(f"❌ 에러 발생: {e}")
    
    finally:
        driver.quit()

    # --- [4. CSV 저장] ---
    if all_links:
        df = pd.DataFrame(all_links, columns=['url'])
        filename = "trump_links_full.csv"
        df.to_csv(filename, index=False, encoding="utf-8-sig")
        print(f"💾 '{filename}' 파일로 저장 완료! (현재 폴더 확인)")
        return df
    else:
        print("⚠️ 저장할 데이터가 없습니다.")
        return None

# 실행
df_full = scrape_all_links()
if df_full is not None:
    print(df_full.head()) # 상위 5개 미리보기

🚀 [전체 수집 모드] 브라우저 실행 중...
🌐 사이트 접속: https://rollcall.com/factbase/trump/search/
⬇️ 전체 데이터를 위해 스크롤을 내립니다...
   ... 스크롤 0회 완료 (현재 높이: 13444)
   ... 스크롤 10회 완료 (현재 높이: 78267)
   ... 스크롤 20회 완료 (현재 높이: 142015)
   ... 스크롤 30회 완료 (현재 높이: 205803)
   ... 스크롤 40회 완료 (현재 높이: 268149)
   ... 스크롤 50회 완료 (현재 높이: 330931)
   ... 스크롤 60회 완료 (현재 높이: 394994)
   ... 스크롤 70회 완료 (현재 높이: 459369)
   ... 스크롤 80회 완료 (현재 높이: 523243)
   ... 스크롤 90회 완료 (현재 높이: 585647)
   ... 스크롤 100회 완료 (현재 높이: 646875)
   ... 스크롤 110회 완료 (현재 높이: 706297)
   ... 스크롤 120회 완료 (현재 높이: 766244)
   ... 스크롤 130회 완료 (현재 높이: 829278)
🎉 더 이상 로딩될 데이터가 없습니다! (바닥 도달)

🔍 전체 HTML 분석 및 링크 추출 중...
✅ 최종 수집 완료: 총 5425개
💾 'trump_links_full.csv' 파일로 저장 완료! (현재 폴더 확인)
                                                 url
0  https://rollcall.com/factbase/trump/transcript...
1  https://rollcall.com/factbase/trump/transcript...
2  https://rollcall.com/factbase/trump/transcript...
3  https://rollcall.com/factbase/trump/transcript...
4  https://rollcall.com/fac

In [28]:
a=pd.DataFrame(df_full)

In [2]:
import pandas as pd 
df=pd.read_csv('/home/hyuksu/teamproject2/data/all_url.csv')
df['url']


0       https://rollcall.com/factbase/trump/transcript...
1       https://rollcall.com/factbase/trump/transcript...
2       https://rollcall.com/factbase/trump/transcript...
3       https://rollcall.com/factbase/trump/transcript...
4       https://rollcall.com/factbase/trump/transcript...
                              ...                        
5420    https://rollcall.com/factbase/trump/transcript...
5421    https://rollcall.com/factbase/trump/transcript...
5422    https://rollcall.com/factbase/trump/transcript...
5423    https://rollcall.com/factbase/trump/transcript...
5424    https://rollcall.com/factbase/trump/transcript...
Name: url, Length: 5425, dtype: str